In [5]:
import numpy as np
import torch

# all_data = np.load('./data/dad/i3d_feat/testing/negative/000831.npy')
# frames
all_data = np.load(f"{'./data/dad/i3d_feat/testing/negative/000831.npy'}")
all_data1 = np.load(f"{'./data/test/i3d_feat/testing/positive/clip_0014.npy'}")



# all_data = np.load(f"{feature_path}")
# all_feat = torch.from_numpy(all_data['data'])[:, 1:, :]
# all_bbox = torch.from_numpy(all_data['det']).float()  #(x1, y1, x2, y2, cls, accident/no acc)bottom left and top right coordinates

# curr_vid_label = int(all_data['labels'][1])  

print(all_data.dtype)
print("all_data")
print(all_data1.dtype)

FileNotFoundError: [Errno 2] No such file or directory: './data/test/i3d_feat/testing/positive/clip_0014.npy'

In [6]:
import torch
import numpy as np
from models import *
#from dataset_dad import *
from torch.utils.data import DataLoader

import argparse

import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image

from torch.optim.lr_scheduler import MultiStepLR

from torchmetrics.functional import pairwise_cosine_similarity
import scipy.io as io

import sklearn 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import time
from eval_utils import evaluation

/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [7]:
torch.manual_seed(0)   #3407

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Classification criterion
cls_criterion = nn.CrossEntropyLoss().to(device)

best_ap = -1 
n_frames = 100

In [9]:
def test_model(epoch, model, test_dataloader):

	""" Function to evaluate the model on the test data 
	Inputs: 
	epoch: training epoch number (0 if it is only testing)
	model: trained model 
	test_dataloader: Dataloader for the testset 
	"""

	global best_ap
	model.eval()
	total_correct, total, all_toa = 0, 0, []
 
	for batch_i, (X, edge_index, y_true, img_feat, video_adj_list, edge_embeddings, temporal_adj_list, obj_vis_feat, batch_vec, toa) in enumerate(test_dataloader):
		print("Batch: ", batch_i)
		X = X.reshape(-1, X.shape[2])
		img_feat = img_feat.reshape(-1, img_feat.shape[2])
		edge_index = edge_index.reshape(-1, edge_index.shape[2])
		edge_embeddings = edge_embeddings.view(-1, edge_embeddings.shape[-1])
		video_adj_list = video_adj_list.reshape(-1, video_adj_list.shape[2])
		temporal_adj_list = temporal_adj_list.reshape(-1, temporal_adj_list.shape[2])
		y = y_true.reshape(-1) 
		
		obj_vis_feat = obj_vis_feat.reshape(-1, obj_vis_feat.shape[-1]).to(device)
		feat_sim = pairwise_cosine_similarity(obj_vis_feat+1e-7, obj_vis_feat+1e-7)
		temporal_edge_w = feat_sim[temporal_adj_list[0, :], temporal_adj_list[1, :]]
		batch_vec = batch_vec.view(-1).long()

		X, edge_index, y, img_feat, video_adj_list = X.to(device), edge_index.to(device), y.to(device), img_feat.to(device), video_adj_list.to(device)
		temporal_adj_list, temporal_edge_w, edge_embeddings, batch_vec = temporal_adj_list.to(device), temporal_edge_w.to(device), edge_embeddings.to(device), batch_vec.to(device)
		all_toa += [toa.item()]
        
		with torch.no_grad():
			logits, probs = model(X, edge_index, img_feat, video_adj_list, edge_embeddings, temporal_adj_list, temporal_edge_w, batch_vec)
		
		pred_labels = probs.argmax(1)
		
		total_correct += (pred_labels == y).cpu().numpy().sum()
		total += y.shape[0]
		
		if batch_i == 0: 
			all_probs_vid2 = probs[:, 1].cpu().unsqueeze(0)
			all_pred = pred_labels.cpu()
			all_y =  y.cpu() #.unsqueeze(0)
			all_y_vid =  torch.max(y).unsqueeze(0).cpu() #y.cpu() #.unsqueeze(0)
		else: 
			all_probs_vid2 = torch.cat((all_probs_vid2, probs[:, 1].cpu().unsqueeze(0)))
			all_pred = torch.cat((all_pred, pred_labels.cpu()))
			all_y = torch.cat((all_y, y.cpu()))
			all_y_vid = torch.cat((all_y_vid, torch.max(y).unsqueeze(0).cpu()))

		# Empty cache
		if torch.cuda.is_available():
			torch.cuda.empty_cache()
	print("All toa :", all_toa)
	#Print the avergae precision 
	print("All Probs vid2 : ", len(all_probs_vid2))
	print("All y vid : ", len(all_y_vid))
	return all_probs_vid2, all_y_vid, all_toa
	
	avg_prec, curr_ttc, _ = evaluation(all_probs_vid2.numpy(), all_y_vid.numpy(), all_toa)    
	avg_prec = 100 * avg_prec

	#Print the confusion matrix 
	cf = confusion_matrix(all_y.numpy(), all_pred.numpy())
	print(cf)

	#class-wise accuracy 
	class_recall = cf.diagonal()/cf.sum(axis=1)
	print(np.round(class_recall, 3))

	if bool(opt.test_only):
		exit(0)

	#Saving checkpoint
	if avg_prec > best_ap:
		best_ap = avg_prec
		os.makedirs("model_checkpoints/dad", exist_ok=True)
		torch.save(model.state_dict(), f"model_checkpoints/dad/{model.__class__.__name__}_{epoch}.pth")
		print(f"Saved the model checkpoint - model_checkpoints/dad/{model.__class__.__name__}_{epoch}.pth")
	print("Best Frame avg precision: %.2f%%" % (best_ap))

	model.train()
	print("")


In [10]:
from dataset_dad import Dataset
img_dataset_path = "data/dad/i3d_feat"
dataset_path = "data/dad/obj_feat"
split_path = "splits_dad/"
obj_mapping_file = "data/dad/obj_idx_to_labels.json"
ref_interval = 20
video_batch_size = 1
# Define training set
train_dataset = Dataset(
    img_dataset_path = img_dataset_path,
    dataset_path=dataset_path,
    split_path=split_path,
    #frame_batch_size=opt.batch_size,
    ref_interval=ref_interval,
    objmap_file=obj_mapping_file,
    training=True,
)
train_dataloader = DataLoader(train_dataset, batch_size=video_batch_size, shuffle=True, num_workers=8)







In [11]:
# Define test set
test_video_batch_size = 1
test_dataset = Dataset(
    img_dataset_path = img_dataset_path,
    dataset_path=dataset_path,
    split_path=split_path,
    #frame_batch_size=opt.batch_size,
    ref_interval=ref_interval,
    objmap_file=obj_mapping_file,
    training=False,
)
test_dataloader = DataLoader(test_dataset, batch_size=test_video_batch_size, shuffle=False, num_workers=8)




In [12]:
	# Define network
input_dim = 4096
embedding_dim = 256
img_feat_dim = 2048
num_classes = 2
model = SpaceTempGoG_detr_dad(input_dim=input_dim, embedding_dim=embedding_dim, img_feat_dim=img_feat_dim, num_classes=num_classes).to(device)
print(model)
	
model.train()

SpaceTempGoG_detr_dad(
  (x_fc): Linear(in_features=4096, out_features=512, bias=True)
  (x_bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (obj_l_fc): Linear(in_features=300, out_features=128, bias=True)
  (obj_l_bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gc1_spatial): GCNConv(640, 128)
  (gc1_norm1): InstanceNorm(128)
  (gc1_temporal): GCNConv(640, 128)
  (gc1_norm2): InstanceNorm(128)
  (pool): TopKPooling(256, ratio=0.8, multiplier=1.0)
  (img_fc): Linear(in_features=2048, out_features=512, bias=True)
  (gc2_sg): GATv2Conv(256, 128, heads=1)
  (gc2_norm1): InstanceNorm(128)
  (gc2_i3d): GATv2Conv(512, 128, heads=1)
  (gc2_norm2): InstanceNorm(128)
  (classify_fc1): Linear(in_features=256, out_features=128, bias=True)
  (classify_fc2): Linear(in_features=128, out_features=2, bias=True)
  (relu): LeakyReLU(negative_slope=0.2)
  (softmax): Softmax(dim=-1)
)


SpaceTempGoG_detr_dad(
  (x_fc): Linear(in_features=4096, out_features=512, bias=True)
  (x_bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (obj_l_fc): Linear(in_features=300, out_features=128, bias=True)
  (obj_l_bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gc1_spatial): GCNConv(640, 128)
  (gc1_norm1): InstanceNorm(128)
  (gc1_temporal): GCNConv(640, 128)
  (gc1_norm2): InstanceNorm(128)
  (pool): TopKPooling(256, ratio=0.8, multiplier=1.0)
  (img_fc): Linear(in_features=2048, out_features=512, bias=True)
  (gc2_sg): GATv2Conv(256, 128, heads=1)
  (gc2_norm1): InstanceNorm(128)
  (gc2_i3d): GATv2Conv(512, 128, heads=1)
  (gc2_norm2): InstanceNorm(128)
  (classify_fc1): Linear(in_features=256, out_features=128, bias=True)
  (classify_fc2): Linear(in_features=128, out_features=2, bias=True)
  (relu): LeakyReLU(negative_slope=0.2)
  (softmax): Softmax(dim=-1)
)

In [13]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params}")

Total trainable parameters: 3581698


In [14]:
	# Add weights from checkpoint model if specified
checkpoint_model ="model_checkpoints/dad_model.pth"
if checkpoint_model:
    #model.load_state_dict(torch.load(checkpoint_model, map_location=torch.device('cpu')))
    #model.load_state_dict(torch.load(checkpoint_model))
    checkpoint = torch.load(checkpoint_model, map_location=torch.device('cpu'))


In [15]:
test_only =1
if bool(test_only):
    all_probs_vid2, all_y_vid, all_toa  = test_model(0, model, test_dataloader)

python(98173) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98178) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(98181) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
python(98202) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python

feature_path:: data/dad/obj_feat/testing/b008_000927.npz
feature_path:: data/dad/obj_feat/testing/b031_001094.npz
feature_path:: data/dad/obj_feat/testing/b001_000860.npz
feature_path:: data/dad/obj_feat/testing/b015_000459.npz
feature_path:: data/dad/obj_feat/testing/b031_000561.npz
feature_path:: data/dad/obj_feat/testing/b004_000903.npz
feature_path:: data/dad/obj_feat/testing/b017_000556.npz
feature_path:: data/dad/obj_feat/testing/b035_001096.npz
feature_path:: data/dad/obj_feat/testing/b025_001008.npz
feature_path:: data/dad/obj_feat/testing/b006_000880.npz
feature_path:: data/dad/obj_feat/testing/b008_000866.npz
feature_path:: data/dad/obj_feat/testing/b010_000879.npz
feature_path:: data/dad/obj_feat/testing/b022_001020.npz
feature_path:: data/dad/obj_feat/testing/b024_000938.npz
feature_path:: data/dad/obj_feat/testing/b035_000589.npz
feature_path:: data/dad/obj_feat/testing/b021_000986.npz
Batch:  0
feature_path:: data/dad/obj_feat/testing/b040_001035.npz
feature_path:: data/d

In [16]:
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
scheduler = MultiStepLR(optimizer, milestones=[25], gamma=0.5)

In [ ]:
time_of_accidents = all_toa
all_labels = all_y_vid.numpy()
all_pred = all_probs_vid2.numpy()
fps=20.0
preds_eval = []
min_pred = np.inf
n_frames = 0
for idx, toa in enumerate(time_of_accidents):
    print("Idx %d, toa %d"%(idx, toa))
    if all_labels[idx] > 0:
        pred = all_pred[idx, :int(toa)]  # positive video
    else:
        pred = all_pred[idx, :]  # negative video
    # find the minimum prediction
    min_pred = np.min(pred) if min_pred > np.min(pred) else min_pred
    preds_eval.append(pred)
    n_frames += len(pred)
total_seconds = all_pred.shape[1] / fps

In [ ]:
# iterate a set of thresholds from the minimum predictions
# temp_shape = int((1.0 - max(min_pred, 0)) / 0.001 + 0.5) 
print("n_frames : ",n_frames)
n_frames = 741
Precision = np.zeros((n_frames))
Recall = np.zeros((n_frames))
Time = np.zeros((n_frames))
cnt = 0
print("N_frames: ", n_frames)
print("Precision: ", Precision.shape)
print("Min Pred : ",min_pred)
print(np.arange(max(min_pred, 0), 1.0, 0.001).shape)
print("Len Pred Eval: ", preds_eval[0].shape)

In [ ]:
for Th in np.arange(max(min_pred, 0), 1.0, 0.001):
    print("TH:",Th)
    Tp = 0.0
    Tp_Fp = 0.0
    Tp_Tn = 0.0
    time = 0.0
    counter = 0.0  # number of TP videos
    # iterate each video sample
    for i in range(len(preds_eval)):
        # true positive frames: (pred->1) * (gt->1)
        tp =  np.where(preds_eval[i]*all_labels[i]>=Th)
        Tp += float(len(tp[0])>0)
        if float(len(tp[0])>0) > 0:
            # if at least one TP, compute the relative (1 - rTTA)
            time += tp[0][0] / float(time_of_accidents[i])
            counter = counter+1
        # all positive frames
        Tp_Fp += float(len(np.where(preds_eval[i]>=Th)[0])>0)
    

    print("TPFP : ",Tp_Fp)
    
    if Tp_Fp == 0:  # predictions of all videos are negative
        continue
    else:
        try:
            Precision[cnt] = Tp/Tp_Fp
        except ValueError as e:
            print("Error : ",e)
    if np.sum(all_labels) ==0: # gt of all videos are negative
        continue
    else:
        Recall[cnt] = Tp/np.sum(all_labels)
    if counter == 0:
        continue
    else:
        Time[cnt] = (1-time/counter)
    cnt += 1


In [ ]:
new_index = np.argsort(Recall)
Precision = Precision[new_index]
Recall = Recall[new_index]
Time = Time[new_index]
print("New index: ", new_index.shape)
print("Precision: ", Precision[new_index].shape)
print("Recall: ", Recall[new_index].shape)
print("Time: ", Time[new_index].shape)

In [ ]:

# unique the recall, and fetch corresponding precisions and TTAs
_,rep_index = np.unique(Recall,return_index=1)
rep_index = rep_index[1:]
new_Time = np.zeros(len(rep_index))
new_Precision = np.zeros(len(rep_index))
for i in range(len(rep_index)-1):
        new_Time[i] = np.max(Time[rep_index[i]:rep_index[i+1]])
        new_Precision[i] = np.max(Precision[rep_index[i]:rep_index[i+1]])
# sort by descending order
new_Time[-1] = Time[rep_index[-1]]
new_Precision[-1] = Precision[rep_index[-1]]
new_Recall = Recall[rep_index]

In [ ]:
AP = 0.0
if new_Recall[0] != 0:
    AP += new_Precision[0]*(new_Recall[0]-0)
for i in range(1,len(new_Precision)):
    AP += (new_Precision[i-1]+new_Precision[i])*(new_Recall[i]-new_Recall[i-1])/2


In [ ]:
# transform the relative mTTA to seconds
mTTA = np.mean(new_Time) * total_seconds
print("Average Precision= %.4f, mean Time to accident= %.4f"%(AP, mTTA))
sort_time = new_Time[np.argsort(new_Recall)]
sort_recall = np.sort(new_Recall)
TTA_R80 = sort_time[np.argmin(np.abs(sort_recall-0.8))] * total_seconds
print("Recall@80%, Time to accident= " +"{:.4}".format(TTA_R80))

In [ ]:
import cv2
import numpy as np

def video_to_npy(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Convert frame to RGB (OpenCV uses BGR by default)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
    
    cap.release()
    
    # Convert list to NumPy array
    frames_array = np.array(frames)
    
    # Save to .npy file
    np.save(output_path, frames_array)
    print(f"Saved frames to {output_path}")

# Example usage
video_path = '/Users/sacithrangana/Desktop/Msc/AAI/text.mp4'
output_npy_path = './output_frames.npy'
video_to_npy(video_path, output_npy_path)


In [ ]:
import cv2
import os

def split_video_to_clips(video_path, output_dir, clip_duration=5, fps=20):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    # Get video properties
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Calculate the number of frames per clip
    clip_frames = int(clip_duration * fps)
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    current_clip = 0
    current_frame = 0
    
    while True:
        frames = []
        for _ in range(clip_frames):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            current_frame += 1
        
        if not frames:
            break
        
        # Save current clip
        clip_filename = os.path.join(output_dir, f"clip_{current_clip:04d}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(clip_filename, fourcc, fps, (width, height))
        
        for frame in frames:
            out.write(frame)
        
        out.release()
        print(f"Saved clip: {clip_filename}")
        
        current_clip += 1
        
        if current_frame >= total_frames:
            break
    
    cap.release()

# Example usage
video_path = '/Users/sacithrangana/Desktop/Msc/AAI/text.mp4'
output_dir = './input_clips/positive'
split_video_to_clips(video_path, output_dir)

video_path = '/Users/sacithrangana/Downloads/video2.mp4'
output_dir = './input_clips/negative'
split_video_to_clips(video_path, output_dir)

In [ ]:
# import cv2
# import numpy as np

# def video_to_npy(video_path, output_path):
#     # Open the video file
#     cap = cv2.VideoCapture(video_path)
    
#     frames = []
    
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break
#         # Convert frame to RGB (OpenCV uses BGR by default)
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         frames.append(frame)
    
#     cap.release()
    
#     # Convert list to NumPy array
#     frames_array = np.array(frames)
    
#     # print(output_path)
#     # Save to .npy file
#     # np.save(output_path, frames_array)
#     print(f"Saved frames to {output_path}")

# Example usage
# video_path_positive = './input_clips/positive'
# video_path_negative = './input_clips/negative'


# # loop through all the clips in the output directory
# for clip in os.listdir(video_path_positive):
#     output_npy_path = './output_frames/positive/'
#     clip_path = os.path.join(output_dir, clip)
#     clip = clip.split(".")[0]
#     output_npy_path = os.path.join(output_npy_path, f"{clip}"   + ".npy")
#     video_to_npy(video_path_positive, output_npy_path)

# for clip in os.listdir(video_path_negative):
#     output_npy_path = './output_frames/nagative/'
#     clip_path = os.path.join(output_dir, clip)
#     clip = clip.split(".")[0]
#     output_npy_path = os.path.join(output_npy_path, f"{clip}"   + ".npy")
#     video_to_npy(video_path_negative, output_npy_path)


In [ ]:
import cv2
import numpy as np
import os
import re
import matplotlib.pyplot as plt

def video_to_3d_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Convert frame to grayscale (2D)
        # gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

def process_videos(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        frames = video_to_3d_frames(video_path)
        output_path = os.path.join(output_folder, os.path.splitext(video_file)[0] + '.npy')
        np.save(output_path, frames)
        print(f"Saved {output_path}")

def plot_frames_from_npy_sorted(output_folder, num_frames=9):
    npy_files = [f for f in os.listdir(output_folder) if f.endswith('.npy')]
    
    # Sort files based on clip number assuming filenames are in format 'clip_<number>.npy'
    npy_files.sort(key=lambda x: int(re.findall(r'\d+', x)[0]))
    
    plt.figure(figsize=(15, 15))
    
    for i in range(min(num_frames, len(npy_files))):
        npy_path = os.path.join(output_folder, npy_files[i])
        frames = np.load(npy_path)
        
        # Plot the first frame of each video
        if frames.size > 0:
            plt.subplot(3, 3, i + 1)
            plt.imshow(frames[0], cmap='gray')
            plt.title(os.path.basename(npy_path))
            plt.axis('off')
    plt.savefig('2d_frames.png')
    plt.show()

# Example usage
input_folder = 'input_clips/positive'
output_folder = 'output_frames/positve'
process_videos(input_folder, output_folder)
plot_frames_from_npy_sorted(output_folder)

input_folder = 'input_clips/negative'
output_folder = 'output_frames/negative'
process_videos(input_folder, output_folder)
plot_frames_from_npy_sorted(output_folder)

In [ ]:
import cv2
import numpy as np
import os
import re
import matplotlib.pyplot as plt

def video_to_3d_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Append the frame as it is (3D: height, width, channels)
        frames.append(frame)
    
    cap.release()
    return np.array(frames)

def process_videos_to_individual_npz(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        frames = video_to_3d_frames(video_path)
        video_key = os.path.splitext(video_file)[0]
        
        # Save each video's frames into a separate .npz file
        output_path = os.path.join(output_folder, f'{video_key}.npz')
        np.savez(output_path, frames=frames)
        print(f"Saved {output_path}")

def plot_frames_from_npz_sorted(output_folder, num_videos=9):
    npz_files = [f for f in os.listdir(output_folder) if f.endswith('.npz')]
    
    # Sort files based on clip number assuming filenames are in format 'clip_<number>.npz'
    npz_files.sort(key=lambda x: int(re.findall(r'\d+', x)[0]))
    
    plt.figure(figsize=(15, 15))
    
    for i in range(min(num_videos, len(npz_files))):
        npz_path = os.path.join(output_folder, npz_files[i])
        data = np.load(npz_path)
        frames = data['frames']
        
        # Plot the first frame of each video
        if frames.size > 0:
            plt.subplot(3, 3, i + 1)
            plt.imshow(cv2.cvtColor(frames[0], cv2.COLOR_BGR2RGB))
            plt.title(os.path.splitext(npz_files[i])[0])
            plt.axis('off')
    
    # plt.savefig('3d_frames.png')
    plt.show()

# Example usage
input_folder = 'input_clips/positive'
output_folder = 'output_npz/positive'
process_videos_to_individual_npz(input_folder, output_folder)

# Plotting frames from the .npz files in sorted order
plot_frames_from_npz_sorted(output_folder)


# Example usage
input_folder = 'input_clips/negative'
output_folder = 'output_npz/negative'
process_videos_to_individual_npz(input_folder, output_folder)

# Plotting frames from the .npz files in sorted order
plot_frames_from_npz_sorted(output_folder)

In [ ]:
# Paths and configurations
video_path = './input_clips/positive/clip_0000.mp4'
vgg16_model = init_vgg16()
det_model = init_mmdet_model('path/to/config_file.py', 'path/to/checkpoint_file.pth')

# Process video
frame_features, box_features, bboxes = process_video(video_path, vgg16_model, det_model)

# Define labels and ID
labels = np.array([0, 1])  # Example labels
video_id = 'video_001'

# Save features to .npz
save_npz('path/to/output_file.npz', frame_features, box_features, bboxes, labels, video_id)


Frame Stats

In [4]:
import cv2
import numpy as np
import os

def main(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    # Video properties
    fps = 20
    duration = 5
    total_frames = fps * duration

    # Initialize list to store statistics
    frame_stats = []

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Append statistics to the list
        frame_stats.append([frame_width, frame_height])

    # Release the video capture object
    cap.release()

    # Convert the list to a 2D numpy array
    frame_stats_array = np.array(frame_stats)

    print(frame_stats_array.shape)  # Should print (100, 2)
    np.save(output_path, frame_stats_array)

if __name__ == '__main__':

    input_folder = 'input_clips/negative'
    output_folder = 'data/test/frames_stats/testing/negative'
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, os.path.splitext(video_file)[0][5:] + '.npy')
        main(video_path, output_path)

    input_folder = 'input_clips/positive'
    output_folder = 'data/test/frames_stats/testing/positive'
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, os.path.splitext(video_file)[0][5:] + '.npy')
        main(video_path, output_path)


(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(11, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(100, 2)
(54, 2)


VG16

In [17]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np
import os

class VGG16FeatureExtractor(nn.Module):
    def __init__(self):
        super(VGG16FeatureExtractor, self).__init__()
        vgg16 = models.vgg16(pretrained=True)
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        self.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1])  # remove the last layer

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

def extract_features_from_video(video_path, extractor, transform, frame_count=50, box_count=19):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < frame_count:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frame = transform(frame)
        frames.append(frame)
    cap.release()

    frames = torch.stack(frames)
    frame_features = extractor(frames)
    frame_level_features = frame_features.unsqueeze(1)  # Add dimension for concatenation

    box_level_features = torch.randn(frame_count, box_count, 4096)  # Dummy data for box-level features
    bounding_boxes = torch.randn(frame_count, box_count, 6)  # Dummy data for bounding boxes

    data = torch.cat([frame_level_features, box_level_features], dim=1)
    return data, bounding_boxes

def main(output_path, positive, frames, video_path):
    model = VGG16FeatureExtractor()
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    data, det = extract_features_from_video(video_path, model, transform, frames)
    if positive == 1:
        labels = np.array([0, 1])  # Example label, adjust as needed
    else:
        labels = np.array([1, 0]) 
    video_id = os.path.basename(video_path)

    # Convert tensors to numpy arrays
    data_np = data.detach().numpy()
    det_np = det.detach().numpy()

    # Save data into .npz file
    np.savez(output_path, data=data_np, det=det_np, labels=labels, ID=video_id)

    print(f'Data shape: {data_np.shape}')  # Should be (50, 20, 4096)
    print(f'Detected bounding boxes shape: {det_np.shape}')  # Should be (50, 19, 6)
    print(f'Labels: {labels}')
    print(f'Video ID: {video_id}')
    print(f'Saved data to {output_path}')

if __name__ == '__main__':

    # def process_videos(input_folder, output_folder):
    # if not os.path.exists(output_folder):
    #     os.makedirs(output_folder)
    input_folder = 'input_clips/negative'
    output_folder = 'data/test/obj_feat/testing'
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
    # for video_file in video_files:
    #     video_path = os.path.join(input_folder, video_file)
    #     print(video_path)
    #     output_path = os.path.join(output_folder, os.path.splitext(video_file)[0][5:] + '.npz')
    #     main(output_path, 0, 100, video_path)
    


In [18]:
main('/Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/Graph-Graph/data/test/obj_feat/testing', 0, 11, '/Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/Graph-Graph/input_clips/negative/clip_0006.mp4')

/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Data shape: (11, 20, 4096)
Detected bounding boxes shape: (11, 19, 6)
Labels: [1 0]
Video ID: clip_0006.mp4
Saved data to /Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/Graph-Graph/data/test/obj_feat/testing


In [12]:
input_folder = 'input_clips/positive'
output_folder = 'data/test/obj_feat/testing'
video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
for video_file in video_files:
    video_path = os.path.join(input_folder, video_file)
    output_path = os.path.join(output_folder, os.path.splitext(video_file)[0][5:]+ '.npz')
    main(output_path, 1, 100, video_path)

Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0016.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0015.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0014.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0010.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0011.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0013.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0007.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0012.mp4
Data shape: (100, 20, 4096)
Detected bounding boxes shape: (100, 19, 6)
Labels: [0 1]
Video ID: clip_0008.mp4


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 54 but got size 100 for tensor number 1 in the list.

In [16]:
main('/Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/Graph-Graph/data/test/obj_feat/testing', 1,54, '/Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/Graph-Graph/input_clips/positive/clip_0009.mp4')

Data shape: (54, 20, 4096)
Detected bounding boxes shape: (54, 19, 6)
Labels: [0 1]
Video ID: clip_0009.mp4


I3D feartures with VG16 - tuvan

In [1]:
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

def main_vgg(video_path, output_path):
    # Load the VGG16 model pre-trained on ImageNet and remove the top layer
    base_model = VGG16(weights='imagenet', include_top=False)
    model = Model(inputs=base_model.input, outputs=base_model.output)
    print("loaded")
    cap = cv2.VideoCapture(video_path)
    print("video loaded")       
    # Define the number of frames and the embedding size
    num_frames = 100
    embedding_size = 2048  # This size is the output shape after the VGG16 layer

    # Create an array to hold the embeddings
    embeddings = np.zeros((num_frames, embedding_size))

    frame_count = 0
    while frame_count < num_frames:
        print("frame:: ", frame_count)   
        ret, frame = cap.read()
        if not ret:
            break
        
        # Preprocess the frame
        frame = cv2.resize(frame, (224, 224))  # Resize to VGG16 expected input size
        frame = frame.astype('float32')
        frame = preprocess_input(frame)
        frame = np.expand_dims(frame, axis=0)

        # Extract features using VGG16
        features = model.predict(frame)
        embeddings[frame_count] = np.squeeze(features).flatten()[:embedding_size]
        
        
        frame_count += 1
    embeddings = embeddings.astype(np.float32)
    # Release the video capture object
    cap.release()

    # Save the embeddings as a .npy file
    np.save(output_path, embeddings)
    print(f"Saved embeddings to {output_npy_path}")

2024-08-26 22:13:17.389778: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os

input_folder = 'input_clips/positive'
output_folder = 'data/test/i3d_feat/testing/positive'
video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
for video_file in video_files:
    video_path = os.path.join(input_folder, video_file)
    output_path = os.path.join(output_folder, os.path.splitext(video_file)[0][5:]+ '.npy')
    main_vgg(video_path, output_path)

loaded
video loaded
frame::  0


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

# Load the VGG16 model pre-trained on ImageNet and remove the top layer
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

# Initialize video capture
video_path = 'clip_1.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Define the number of frames and the embedding size
num_frames = 100
embedding_size = 2048  # This size is the output shape after the VGG16 layer

# Create an array to hold the embeddings
embeddings = np.zeros((num_frames, embedding_size))

frame_count = 0
while frame_count < num_frames:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the frame
    frame = cv2.resize(frame, (224, 224))  # Resize to VGG16 expected input size
    frame = frame.astype('float32')
    frame = preprocess_input(frame)
    frame = np.expand_dims(frame, axis=0)

    # Extract features using VGG16
    features = model.predict(frame)
    embeddings[frame_count] = np.squeeze(features).flatten()[:embedding_size]
    
    
    frame_count += 1
embeddings = embeddings.astype(np.float32)
# Release the video capture object
cap.release()

# Save the embeddings as a .npy file
np.save('embeddings.npy', embeddings)

print("Embeddings saved to 'embeddings.npy'")

I3D

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import cv2
import numpy as np

def main(output_path):
    # Initialize VGG16 model
    model = models.vgg16(pretrained=True).features
    model.eval()

    # Define preprocessing transform
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load video
    video_path = './input_clips/negative/clip_0000.mp4'
    cap = cv2.VideoCapture(video_path)

    # Collect frames
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    # Process frames
    processed_frames = [preprocess(frame) for frame in frames]
    frame_tensor = torch.stack(processed_frames)

    # Extract features
    with torch.no_grad():
        features = model(frame_tensor).view(len(frames), -1)

    # Limit features to (100, 2048)
    if features.shape[0] > 100:
        features = features[:100]
    elif features.shape[0] < 100:
        # Handle case where there are fewer than 100 frames
        padding = torch.zeros(100 - features.shape[0], features.shape[1])
        features = torch.cat([features, padding])

    # Resize feature vector to (2048)
    features = features[:, :2048]

    # Convert features to numpy array
    features_np = features.numpy()

    np.save(output_path, features_np)

    print("Features shape:", features_np.shape)


if __name__ == '__main__':

    input_folder = 'input_clips/positive'
    output_folder = 'data/test/i3d_feat/testing/positive'
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
        
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, os.path.splitext(video_file)[0]+ '.npy')
        main(output_path)

    input_folder = 'input_clips/negative'
    output_folder = 'data/test/i3d_feat/testing/negative'
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
        
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, os.path.splitext(video_file)[0] + '.npy')
        main(output_path)


VG16 New

In [ ]:
# import torch
# import torchvision
# import cv2
# import numpy as np

# # Load pre-trained VGG-16 model
# vgg16 = torchvision.models.vgg16(pretrained=True)
# vgg16.eval()

# # Modify VGG-16 to use as a feature extractor
# feature_extractor = torch.nn.Sequential(*list(vgg16.features.children()))

# # Load a pre-trained Faster R-CNN model for object detection
# faster_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# faster_rcnn.eval()


# def extract_frames(video_path):
#     vidcap = cv2.VideoCapture(video_path)
#     frames = []
#     success, image = vidcap.read()
#     while success:
#         frames.append(image)
#         success, image = vidcap.read()
#     vidcap.release()
#     return frames

# video_path = './input_clips/positive/clip_0007.mp4'
# frames = extract_frames(video_path)


# def preprocess_frame(frame):
#     # Convert BGR (OpenCV) to RGB
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     # Resize to 224x224 as expected by VGG-16
#     frame_resized = cv2.resize(frame_rgb, (224, 224))
#     # Convert to tensor and normalize
#     frame_tensor = torchvision.transforms.functional.to_tensor(frame_resized)
#     frame_tensor = torchvision.transforms.functional.normalize(frame_tensor, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     return frame_tensor

# preprocessed_frames = [preprocess_frame(frame) for frame in frames]



# # Convert list of frames to batch tensor
# batch_tensor = torch.stack(preprocessed_frames)

# # Extract features using VGG-16
# with torch.no_grad():
#     features = feature_extractor(batch_tensor)

# # Perform object detection using Faster R-CNN
# with torch.no_grad():
#     detections = faster_rcnn(batch_tensor)

# # Display detections (for the first frame as an example)
# import matplotlib.pyplot as plt

# def display_detections(image, detections):
#     for box, score, label in zip(detections['boxes'], detections['scores'], detections['labels']):
#         if score > 0.5:  # Adjust threshold as needed
#             x1, y1, x2, y2 = box
#             cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
#     plt.imshow(image)
#     plt.show()

# # Display detections for the first frame
# display_detections(frames[0], detections[0])




# Load a pre-trained object detection model
detection_model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()

# Define transformation
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features_and_draw_boxes(video_path, positivity=True, frame_count=100):
    cap = cv2.VideoCapture(video_path)
    # frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_features = []
    bounding_boxes = []  # To store bounding boxes (x1, y1, x2, y2, prob, cls)
    one_hot_labels = []  # To store one-hot labels

    for _ in range(frame_count):
        print(_)
        ret, frame = cap.read()
        if not ret:
            break
        
        # Object detection
        input_tensor = preprocess(frame).unsqueeze(0)
        with torch.no_grad():
            detection = detection_model([input_tensor.squeeze()])[0]
        
        # Draw bounding boxes on the frame
        boxes = detection['boxes'].cpu().numpy()
        scores = detection['scores'].cpu().numpy()
        labels = detection['labels'].cpu().numpy()
        
        # Filter boxes with a score threshold (e.g., 0.5)
        indices = np.where(scores > 0.5)[0]
        filtered_boxes = boxes[indices]
        filtered_scores = scores[indices]
        filtered_labels = labels[indices]
        
        for i in range(len(filtered_boxes)):
            box = filtered_boxes[i]
            score = filtered_scores[i]
            label = filtered_labels[i]
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), 2)
            cv2.putText(frame, f'{label}: {score:.2f}', (int(box[0]), int(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        # Save the frame with drawn bounding boxes
        cv2.imwrite(f'frame_with_boxes_{_}.jpg', frame)
        
        # Frame-level feature extraction
        frame_input_tensor = preprocess(frame).unsqueeze(0)
        with torch.no_grad():
            frame_feature = vgg16(frame_input_tensor).numpy().flatten()
        frame_features.append(frame_feature)

        # Store bounding boxes and labels
        boxes_with_scores_labels = np.hstack([filtered_boxes, filtered_scores[:, np.newaxis], filtered_labels[:, np.newaxis]])
        bounding_boxes.append(boxes_with_scores_labels)

        label = [0, 1] if positivity else [1, 0]
        one_hot_labels.append(label)

    cap.release()

    frame_features = np.array(frame_features)
    bounding_boxes = np.array(bounding_boxes)
    one_hot_labels = np.array(one_hot_labels)

    # Shape the features
    frame_level_features = frame_features.reshape(-1, 1, 4096)
    box_level_features = np.tile(frame_features, (1, 19)).reshape(-1, 19, 4096)
    data = np.concatenate((frame_level_features, box_level_features), axis=1)

    return {
        'data': data,
        'det': bounding_boxes,
        'labels': one_hot_labels,
        'ID': video_path.split('/')[-1]
    }

# Example usage
video_path = './input_clips/positive/clip_0007.mp4'
features = extract_features_and_draw_boxes(video_path)
print(features)


In [ ]:
import torch
import torchvision
import cv2
import numpy as np

# Load pre-trained VGG-16 model
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16.eval()

# Modify VGG-16 to use as a feature extractor
feature_extractor = torch.nn.Sequential(*list(vgg16.features.children()))

# Load a pre-trained Faster R-CNN model for object detection
faster_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn.eval()


def extract_frames(video_path):
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    success, image = vidcap.read()
    while success:
        frames.append(image)
        success, image = vidcap.read()
    vidcap.release()
    return frames

video_path = './input_clips/positive/clip_0007.mp4'
frames = extract_frames(video_path)


def preprocess_frame(frame):
    # Convert BGR (OpenCV) to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Resize to 224x224 as expected by VGG-16
    frame_resized = cv2.resize(frame_rgb, (224, 224))
    # Convert to tensor and normalize
    frame_tensor = torchvision.transforms.functional.to_tensor(frame_resized)
    frame_tensor = torchvision.transforms.functional.normalize(frame_tensor, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    return frame_tensor

preprocessed_frames = [preprocess_frame(frame) for frame in frames]



# Convert list of frames to batch tensor
batch_tensor = torch.stack(preprocessed_frames)

# Extract features using VGG-16
with torch.no_grad():
    features = feature_extractor(batch_tensor)

# Perform object detection using Faster R-CNN
with torch.no_grad():
    detections = faster_rcnn(batch_tensor)

# Display detections (for the first frame as an example)
import matplotlib.pyplot as plt

def display_detections(image, detections):
    for box, score, label in zip(detections['boxes'], detections['scores'], detections['labels']):
        if score > 0.5:  # Adjust threshold as needed
            x1, y1, x2, y2 = box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(image)
    plt.show()

# Display detections for the first frame
display_detections(frames[0], detections[0])

In [ ]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load Video
cap = cv2.VideoCapture("./input_clips/positive/clip_0007.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing information on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Image", frame)

    key = cv2.waitKey(1)
    if key == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
Experiment

In [7]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Load the pre-trained model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()  # Set the model to evaluation mode


/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/graphofgraph3.9_new_1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [31]:
import cv2
import numpy as np
from PIL import Image

custom_labels = [
    "person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
    "trafficlight", "firehydrant", "streetsign", "stopsign", "parkingmeter", "bench", "bird",
    "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "hat",
    "backpack", "umbrella", "shoe", "eyeglasses", "handbag", "tie", "suitcase", "frisbee",
    "skis", "snowboard", "sportsball", "kite", "baseballbat", "baseballglove", "skateboard",
    "surfboard", "tennisracket", "bottle", "plate", "wineglass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hotdog", "pizza",
    "donut", "cake", "chair", "sofa", "pottedplant", "bed", "mirror", "diningtable", "window",
    "desk", "toilet", "door", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cellphone",
    "microwave", "oven", "toaster", "sink", "refrigerator", "blender", "book", "clock", "vase",
    "scissors", "teddybear", "hairdrier", "toothbrush", "hairbrush", "Van", "Truck", "Pedestrian",
    "Person_sitting", "Cyclist", "Tram", "Misc", "DontCare"
]

# COCO class IDs used by Faster R-CNN (1-91, but you may need to map only those used in your custom set)
coco_to_custom_mapping = {
    1: "person", 2: "bicycle", 3: "car", 4: "motorbike", 5: "aeroplane", 6: "bus", 
    7: "train", 8: "truck", 9: "boat", 10: "trafficlight", 11: "firehydrant", 12: "streetsign",
    13: "stopsign", 14: "parkingmeter", 15: "bench", 16: "bird", 17: "cat", 18: "dog",
    19: "horse", 20: "sheep", 21: "cow", 22: "elephant", 23: "bear", 24: "zebra", 25: "giraffe",
    26: "hat", 27: "backpack", 28: "umbrella", 29: "shoe", 30: "eyeglasses", 31: "handbag",
    32: "tie", 33: "suitcase", 34: "frisbee", 35: "skis", 36: "snowboard", 37: "sportsball",
    38: "kite", 39: "baseballbat", 40: "baseballglove", 41: "skateboard", 42: "surfboard",
    43: "tennisracket", 44: "bottle", 45: "plate", 46: "wineglass", 47: "cup", 48: "fork",
    49: "knife", 50: "spoon", 51: "bowl", 52: "banana", 53: "apple", 54: "sandwich",
    55: "orange", 56: "broccoli", 57: "carrot", 58: "hotdog", 59: "pizza", 60: "donut",
    61: "cake", 62: "chair", 63: "sofa", 64: "pottedplant", 65: "bed", 66: "mirror",
    67: "diningtable", 68: "window", 69: "desk", 70: "toilet", 71: "door", 72: "tvmonitor",
    73: "laptop", 74: "mouse", 75: "remote", 76: "keyboard", 77: "cellphone", 78: "microwave",
    79: "oven", 80: "toaster", 81: "sink", 82: "refrigerator", 83: "blender", 84: "book",
    85: "clock", 86: "vase", 87: "scissors", 88: "teddybear", 89: "hairdrier", 90: "toothbrush",
    91: "hairbrush"
}




# Function to preprocess image
def preprocess_image(frame):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    transform = T.Compose([
        T.ToTensor(),
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

def init_vgg16(is_acc, video_path, output_path):

    # Load video
    # video_path = './input_clips/positive/clip_0007.mp4'
    cap = cv2.VideoCapture(video_path)

    # Process frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        image_tensor = preprocess_image(frame)

        # Perform object detection
        with torch.no_grad():
            predictions = model(image_tensor)
        label_names = [];
        # Draw bounding boxes
        boxes = predictions[0]['boxes'].cpu().numpy()
        scores = predictions[0]['scores'].cpu().numpy()
        labels = predictions[0]['labels'].cpu().numpy()
        
        det = []
        id = []
        frame_features = []
        box_features = []
        

        # Extract frame-level features (using a CNN, here we just flatten the frame as a placeholder)
        frame_feature = cv2.resize(frame, (224, 224)).flatten()[:4096]  # Resize to ensure it fits 4096 elements
        frame_features.append(frame_feature)

        if len(boxes) > 19:
            boxes = boxes[:19]  # Ensure only 19 boxes are considered
            scores = scores[:19]
            labels  = labels[:19]

        # for box, score, label in zip(boxes, scores, labels):
        #     if score > 0.5 and int(label) in coco_to_custom_mapping:  # Filter for road-specific labels
        #         x1, y1, x2, y2 = map(int, box)
        #         box_frame = frame[y1:y2, x1:x2]
        #         coco_label_id = int(label)
        #         det.append([x1, y1, x2, y2, score, coco_label_id])
        for box in boxes:
            x1, y1, x2, y2, _, _ = box
            box_frame = frame[y1:y2, x1:x2]
            if box_frame.size > 0:
                box_feature = cv2.resize(box_frame, (64, 64)).flatten()[:4096]
                box_features.append(box_feature)
            else:
                box_features.append(np.zeros(4096))  # In case of invalid box

        while len(frame_features) < 100:
            frame_features.append(np.zeros(4096))

        video_name = video_path.split('/')[-1].split('.')[0]
        labels = np.array(is_acc)
        id = np.array(video_name)

        # # Combine frame-level and box-level features
        data = np.concatenate((np.array(frame_features), np.array(box_features)), axis=1)
    np.savez_compressed(output_path + video_name + '.npz', data=data, det=det, labels=labels, id=id)
    cap.release()
    cv2.destroyAllWindows()

    return data, det, labels, id

# Example usage
input_folder = 'input_clips/positive'
output_folder = 'output_npz/positive'


init_vgg16([0,1], "./input_clips/positive/clip_0007.mp4", "./data/test/experiment/data/")

# Example usage
input_folder = 'input_clips/negative'
output_folder = 'output_npz/negative'







AxisError: axis 1 is out of bounds for array of dimension 1

: 

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np
import os

class VGG16FeatureExtractor(nn.Module):
    def __init__(self):
        super(VGG16FeatureExtractor, self).__init__()
        vgg16 = models.vgg16(pretrained=True)
        self.features = vgg16.features
        self.avgpool = vgg16.avgpool
        self.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1])  # remove the last layer

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Function to preprocess image
def preprocess_image(frame):
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    transform = T.Compose([
        T.ToTensor(),
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension


def extract_features_from_video(video_path, extractor, frame_count=50, box_count=19):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < frame_count:
        ret, frame = cap.read()
        if not ret:
            break
        frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame = cv2.resize(frame, (224, 224))
        transform = T.Compose([
        T.ToTensor(),
        ])
        frames.append(transform(frame).unsqueeze(0))
    cap.release()


# (x1, y1, x2, y2, cls, accident/no acc)bottom left and top right coordinates
    # frames = torch.stack(frames)
    # frame_features = extractor(frames)
    # frame_level_features = frame_features.unsqueeze(1)  # Add dimension for concatenation

    # box_level_features = torch.randn(frame_count, box_count, 4096)  # Dummy data for box-level features
    # bounding_boxes = torch.randn(frame_count, box_count, 6)  # Dummy data for bounding boxes

    data = torch.cat([frame_level_features, box_level_features], dim=1)
    return data, bounding_boxes

def main(output_path, positive, frames):
    video_path = './input_clips/negative/clip_0000.mp4'
    # model = VGG16FeatureExtractor()
    # transform = transforms.Compose([
    #     transforms.ToPILImage(),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # ])

    data, det = extract_features_from_video(video_path, model, frames)
    labels = np.array([0, positive])  # Example label, adjust as needed
    video_id = os.path.basename(video_path)

    # Convert tensors to numpy arrays
    data_np = data.detach().numpy()
    det_np = det.detach().numpy()

    # Save data into .npz file
    np.savez(output_path, data=data_np, det=det_np, labels=labels, ID=video_id)

    print(f'Data shape: {data_np.shape}')  # Should be (100, 20, 4096)
    print(f'Detected bounding boxes shape: {det_np.shape}')  # Should be (50, 19, 6)
    print(f'Labels: {labels}')
    print(f'Video ID: {video_id}')

if __name__ == '__main__':

    # def process_videos(input_folder, output_folder):
    # if not os.path.exists(output_folder):
    #     os.makedirs(output_folder)
    input_folder = 'input_clips/negative'
    output_folder = 'data/test/obj_feat/testing'
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, os.path.splitext(video_file)[0] + '.npz')
        main(output_path, 0, 100)
    


In [ ]:
import cv2
import numpy as np

# Function to load YOLO model
def load_yolo_model(cfg_path, weights_path, names_path):
    net = cv2.dnn.readNet(weights_path, cfg_path)
    with open(names_path, 'r') as f:
        classes = f.read().strip().split('\n')
    return net, classes

# Function to perform object detection on a single frame
def detect_objects(net, classes, frame, conf_threshold=0.5, nms_threshold=0.4):
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    detections = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []
    h, w = frame.shape[:2]

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                width = int(detection[2] * w)
                height = int(detection[3] * h)
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                boxes.append([x, y, x + width, y + height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    filtered_boxes = []
    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            filtered_boxes.append(box + [confidences[i], class_ids[i]])

    return filtered_boxes

# Function to process video and extract features and detections
def process_video(video_path, yolo_model, classes):
    cap = cv2.VideoCapture(video_path)
    frame_features = []
    detections = []
    frame_ids = []
    labels = []

    frame_count = 0

    while cap.isOpened() and frame_count < 50:
        ret, frame = cap.read()
        if not ret:
            break

        # Extract frame-level features (using a CNN, here we just flatten the frame as a placeholder)
        frame_feature = cv2.resize(frame, (224, 224)).flatten()[:4096]  # Resize to ensure it fits 4096 elements
        frame_features.append(frame_feature)

        # Detect objects
        boxes = detect_objects(yolo_model, classes, frame)
        if len(boxes) > 19:
            boxes = boxes[:19]  # Ensure only 19 boxes are considered

        # Create box-level features (using the same placeholder approach for now)
        box_features = []
        for box in boxes:
            x1, y1, x2, y2, _, _ = box
            box_frame = frame[y1:y2, x1:x2]
            if box_frame.size > 0:
                box_feature = cv2.resize(box_frame, (64, 64)).flatten()[:4096]
                box_features.append(box_feature)
            else:
                box_features.append(np.zeros(4096))  # In case of invalid box

        while len(box_features) < 19:
            box_features.append(np.zeros(4096))  # Pad with zero features if less than 19 boxes

        detections.append(boxes)

        # Assign a label (placeholder, replace with actual logic)
        label = [1, 0] if frame_count % 2 == 0 else [0, 1]  # Replace with actual condition
        labels.append(label)

        # Collect frame IDs (placeholder, replace with actual video name logic)
        frame_ids.append(f'frame_{frame_count}')  # Replace with actual frame ID or video ID logic

        frame_count += 1

    cap.release()

    # Ensure we have exactly 50 frames
    while len(frame_features) < 50:
        frame_features.append(np.zeros(4096))
        detections.append([[0, 0, 0, 0, 0.0, -1]] * 19)
        labels.append([0, 1])
        frame_ids.append(f'frame_{len(frame_features)}')

    # Convert lists to numpy arrays
    frame_features = np.array(frame_features).reshape((50, 1, 4096))
    box_features = np.array([np.array(f) for f in detections]).reshape((50, 19, 4096))
    detections = np.array(detections)
    labels = np.array(labels)
    frame_ids = np.array(frame_ids)

    # Combine frame-level and box-level features
    combined_features = np.concatenate((frame_features, box_features), axis=1)

    return combined_features, detections, labels, frame_ids

# Paths to YOLO files
cfg_path = 'yolov3.cfg'
weights_path = 'yolov3.weights'
names_path = 'coco.names'

# Load YOLO model
yolo_model, classes = load_yolo_model(cfg_path, weights_path, names_path)

# Process the video
video_path = 'input_video.mp4'
data, det, labels, ID = process_video(video_path, yolo_model, classes)

# Save to .npz file
np.savez('output_data.npz', data=data, det=det, labels=labels, ID=ID)

print("NPZ file successfully created.")

In [ ]:
Frames

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import cv2
import os, sys
import os.path as osp
import argparse
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
        self.dim_feat = 4096
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

ModuleNotFoundError: No module named 'cv2'

In [8]:
def init_feature_extractor(backbone='vgg16', device=torch.device('cuda')):
    feat_extractor = None
    if backbone == 'vgg16':
        feat_extractor = VGG16()
        feat_extractor = feat_extractor.to(device=device)
        feat_extractor.eval()
    else:
        raise NotImplementedError
    return feat_extractor

In [1]:
from mmdet.apis import init_detector, inference_detector, show_result
import mmcv


AssertionError: MMCV==2.2.0 is used but incompatible. Please install mmcv>=1.3.17, <=1.8.0.

In [ ]:
def init_feature_extractor(backbone='vgg16', device=torch.device('cuda')):
    feat_extractor = None
    if backbone == 'vgg16':
        feat_extractor = VGG16()
        feat_extractor = feat_extractor.to(device=device)
        feat_extractor.eval()
    else:
        raise NotImplementedError
    return feat_extractor

In [ ]:
device = torch.device('cuda:'+str(p.gpu_id)) if torch.cuda.is_available() else torch.device('cpu')
feat_extractor = init_feature_extractor(backbone='vgg16', device=device)

In [ ]:
def bbox_sampling(bbox_result, nbox=19, imsize=None, topN=5):
    """
    imsize[0]: height
    imsize[1]: width
    """
    assert not isinstance(bbox_result, tuple)
    bboxes = np.vstack(bbox_result)  # n x 5
    labels = [np.full(bbox.shape[0], i, dtype=np.int32) for i, bbox in enumerate(bbox_result)]
    labels = np.concatenate(labels)  # n
    ndet = bboxes.shape[0]

    # fix bbox 
    new_boxes = []
    for box, label in zip(bboxes, labels):
        x1 = min(max(0, int(box[0])), imsize[1])
        y1 = min(max(0, int(box[1])), imsize[0])
        x2 = min(max(x1 + 1, int(box[2])), imsize[1])
        y2 = min(max(y1 + 1, int(box[3])), imsize[0])
        if (y2 - y1 + 1 > 2) and (x2 - x1 + 1 > 2):
            new_boxes.append([x1, y1, x2, y2, box[4], label])
    if len(new_boxes) == 0:  # no bboxes
        new_boxes.append([0, 0, imsize[1]-1, imsize[0]-1, 1.0, 0])
    new_boxes = np.array(new_boxes, dtype=int)
    # sampling
    n_candidate = min(topN, len(new_boxes))
    if len(new_boxes) <= nbox - n_candidate:
        indices = np.random.choice(n_candidate, nbox - len(new_boxes), replace=True)
        sampled_boxes = np.vstack((new_boxes, new_boxes[indices]))
    elif len(new_boxes) > nbox - n_candidate and len(new_boxes) <= nbox:
        indices = np.random.choice(n_candidate, nbox - len(new_boxes), replace=False)
        sampled_boxes = np.vstack((new_boxes, new_boxes[indices]))
    else:
        sampled_boxes = new_boxes[:nbox]
    
    return sampled_boxes

In [ ]:
def bbox_to_imroi(transform, bboxes, image):
    imroi_data = []
    for bbox in bboxes:
        imroi = image[bbox[1]:bbox[3], bbox[0]:bbox[2], :]
        imroi = transform(Image.fromarray(imroi))  # (3, 224, 224), torch.Tensor
        imroi_data.append(imroi)
    imroi_data = torch.stack(imroi_data)
    return imroi_data

In [ ]:
def extract_features(detector, feat_extractor, video_file, n_frames=100, n_boxes=19):
    assert os.path.join(video_file), video_file
    # prepare video reader and data transformer
    videoReader = mmcv.VideoReader(video_file)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()]
    )
    features = np.zeros((n_frames, n_boxes + 1, feat_extractor.dim_feat), dtype=np.float32)
    detections = np.zeros((n_frames, n_boxes, 6))  # (50 x 19 x 6)
    frame_prev = None
    for idx in range(n_frames):
        if idx >= len(videoReader):
            print("Copy frame from previous time step.")
            frame = frame_prev.copy()
        else:
            frame = videoReader.get_frame(idx)
        # run object detection inference
        bbox_result = inference_detector(detector, frame)
        # sampling a fixed number of bboxes
        bboxes = bbox_sampling(bbox_result, nbox=n_boxes, imsize=frame.shape[:2])
        detections[idx, :, :] = bboxes
        # prepare frame data
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        with torch.no_grad():
            # bboxes to roi feature
            ims_roi = bbox_to_imroi(transform, bboxes, frame)
            ims_roi = ims_roi.float().to(device=device)
            feature_roi = feat_extractor(ims_roi)
            # extract image feature
            ims_frame = transform(Image.fromarray(frame))
            ims_frame = torch.unsqueeze(ims_frame, dim=0).float().to(device=device)
            feature_frame = feat_extractor(ims_frame)
        # obtain feature matrix
        features[idx, 0, :] = np.squeeze(feature_frame.cpu().numpy()) if feature_frame.is_cuda else np.squeeze(feature_frame.detach().numpy())
        features[idx, 1:, :] = np.squeeze(feature_roi.cpu().numpy()) if feature_roi.is_cuda else np.squeeze(feature_roi.detach().numpy())
        frame_prev = frame
    return detections, features

In [ ]:
cfg_file = osp.join("lib/mmdetection", "/Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/supporting-libs/mmdetection/configs/cascade_rcnn_x101_64x4d_fpn_1x_kitti2d.py")
model_file = osp.join("lib/mmdetection", "/Users/sacithrangana/Desktop/Msc/AAI/gitrepo/Graph_Code/supporting-libs/mmdetection/work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_kitti2d/latest.pth")
detector = init_detector(cfg_file, model_file, device=device)
detections, features = extract_features(detector, feat_extractor, "data/test/manipulation", n_frames=100)
feat_file = "1234"[:-4] + '_feature.npz'
np.savez_compressed(feat_file, data=features, det=detections)

In [ ]:
import cv2
import numpy as np

# Function to load YOLO model
def load_yolo_model(cfg_path, weights_path, names_path):
    net = cv2.dnn.readNet(weights_path, cfg_path)
    with open(names_path, 'r') as f:
        classes = f.read().strip().split('\n')
    return net, classes

# Function to perform object detection on a single frame
def detect_objects(net, classes, frame, conf_threshold=0.5, nms_threshold=0.4):
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    detections = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []
    h, w = frame.shape[:2]

    for output in detections:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * w)
                center_y = int(detection[1] * h)
                width = int(detection[2] * w)
                height = int(detection[3] * h)
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                boxes.append([x, y, x + width, y + height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    filtered_boxes = []
    if len(indices) > 0:
        for i in indices.flatten():
            box = boxes[i]
            filtered_boxes.append(box + [confidences[i], class_ids[i]])

    return filtered_boxes

# Function to process video and extract features and detections
def process_video(video_path, yolo_model, classes):
    cap = cv2.VideoCapture(video_path)
    frame_features = []
    detections = []
    frame_ids = []
    labels = []

    frame_count = 0

    while cap.isOpened() and frame_count < 50:
        ret, frame = cap.read()
        if not ret:
            break

        # Extract frame-level features (using a CNN, here we just flatten the frame as a placeholder)
        frame_feature = cv2.resize(frame, (224, 224)).flatten()[:4096]  # Resize to ensure it fits 4096 elements
        frame_features.append(frame_feature)

        # Detect objects
        boxes = detect_objects(yolo_model, classes, frame)
        if len(boxes) > 19:
            boxes = boxes[:19]  # Ensure only 19 boxes are considered

        # Create box-level features (using the same placeholder approach for now)
        box_features = []
        for box in boxes:
            x1, y1, x2, y2, _, _ = box
            box_frame = frame[y1:y2, x1:x2]
            if box_frame.size > 0:
                box_feature = cv2.resize(box_frame, (64, 64)).flatten()[:4096]
                box_features.append(box_feature)
            else:
                box_features.append(np.zeros(4096))  # In case of invalid box

        while len(box_features) < 19:
            box_features.append(np.zeros(4096))  # Pad with zero features if less than 19 boxes

        detections.append(boxes)

        # Assign a label (placeholder, replace with actual logic)
        label = [1, 0] if frame_count % 2 == 0 else [0, 1]  # Replace with actual condition
        labels.append(label)

        # Collect frame IDs (placeholder, replace with actual video name logic)
        frame_ids.append(f'frame_{frame_count}')  # Replace with actual frame ID or video ID logic

        frame_count += 1

    cap.release()

    # Ensure we have exactly 50 frames
    while len(frame_features) < 50:
        frame_features.append(np.zeros(4096))
        detections.append([[0, 0, 0, 0, 0.0, -1]] * 19)
        labels.append([0, 1])
        frame_ids.append(f'frame_{len(frame_features)}')

    # Convert lists to numpy arrays
    frame_features = np.array(frame_features).reshape((50, 1, 4096))
    box_features = np.array([np.array(f) for f in detections]).reshape((50, 19, 4096))
    detections = np.array(detections)
    labels = np.array(labels)
    frame_ids = np.array(frame_ids)

    # Combine frame-level and box-level features
    combined_features = np.concatenate((frame_features, box_features), axis=1)

    return combined_features, detections, labels, frame_ids

# Paths to YOLO files
cfg_path = 'yolov3.cfg'
weights_path = 'yolov3.weights'
names_path = 'coco.names'

# Load YOLO model
yolo_model, classes = load_yolo_model(cfg_path, weights_path, names_path)

# Process the video
video_path = 'input_video.mp4'
data, det, labels, ID = process_video(video_path, yolo_model, classes)

# Save to .npz file
np.savez('output_data.npz', data=data, det=det, labels=labels, ID=ID)

print("NPZ file successfully created.")

Q1

In [15]:
import numpy as np

# Define the grid world
gamma = 0.9
reward = np.array([
    [-0.04, -0.04, -0.04, +1],
    [-0.04, None, -0.04, -1],
    [-0.04, -0.04, -0.04, -0.04]
])

# Possible actions
actions = ['up', 'down', 'left', 'right']
action_vectors = {
    'up': (0, 1),
    'down': (0, -1),
    'left': (-1, 0),
    'right': (1, 0)
}

# Utility initialization
U = np.zeros(reward.shape)

# Value iteration function
def value_iteration(U, reward, gamma, theta=1e-4):
    while True:
        delta = 0
        U_new = np.copy(U)
        for x in range(reward.shape[0]):
            for y in range(reward.shape[1]):
                if reward[x, y] is None:
                    continue
                if (x, y) in [(3, 2), (3, 3)]:
                    U_new[x, y] = reward[x, y]
                else:
                    v = []
                    for action in actions:
                        dx, dy = action_vectors[action]
                        nx, ny = x + dx, y + dy
                        if 0 <= nx < reward.shape[0] and 0 <= ny < reward.shape[1] and reward[nx, ny] is not None:
                            intended = 0.8 * U[nx, ny]
                        else:
                            intended = 0.8 * U[x, y]
                        
                        perp1 = perp2 = 0
                        if action in ['up', 'down']:
                            for a in ['left', 'right']:
                                dx, dy = action_vectors[a]
                                nx, ny = x + dx, y + dy
                                if 0 <= nx < reward.shape[0] and 0 <= ny < reward.shape[1] and reward[nx, ny] is not None:
                                    perp1 += 0.1 * U[nx, ny]
                                else:
                                    perp1 += 0.1 * U[x, y]
                        else:
                            for a in ['up', 'down']:
                                dx, dy = action_vectors[a]
                                nx, ny = x + dx, y + dy
                                if 0 <= nx < reward.shape[0] and 0 <= ny < reward.shape[1] and reward[nx, ny] is not None:
                                    perp2 += 0.1 * U[nx, ny]
                                else:
                                    perp2 += 0.1 * U[x, y]
                        v.append(reward[x, y] + gamma * (intended + perp1 + perp2))
                    U_new[x, y] = max(v)
                delta = max(delta, abs(U_new[x, y] - U[x, y]))
        U = U_new
        if delta < theta:
            break
    return U

# Calculate utilities
U = value_iteration(U, reward, gamma)

# Print utility table
print("Utility Table:")
print(U)

# Determine optimal policy
policy = np.full(reward.shape, None)
for x in range(reward.shape[0]):
    for y in range(reward.shape[1]):
        if reward[x, y] is None or (x, y) in [(3, 2), (3, 3)]:
            continue
        best_action = None
        best_value = -float('inf')
        for action in actions:
            dx, dy = action_vectors[action]
            nx, ny = x + dx, y + dy
            if 0 <= nx < reward.shape[0] and 0 <= ny < reward.shape[1] and reward[nx, ny] is not None:
                intended = 0.8 * U[nx, ny]
            else:
                intended = 0.8 * U[x, y]

            perp1 = perp2 = 0
            if action in ['up', 'down']:
                for a in ['left', 'right']:
                    dx, dy = action_vectors[a]
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < reward.shape[0] and 0 <= ny < reward.shape[1] and reward[nx, ny] is not None:
                        perp1 += 0.1 * U[nx, ny]
                    else:
                        perp1 += 0.1 * U[x, y]
            else:
                for a in ['up', 'down']:
                    dx, dy = action_vectors[a]
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < reward.shape[0] and 0 <= ny < reward.shape[1] and reward[nx, ny] is not None:
                        perp2 += 0.1 * U[nx, ny]
                    else:
                        perp2 += 0.1 * U[x, y]
            value = reward[x, y] + gamma * (intended + perp1 + perp2)
            if value > best_value:
                best_value = value
                best_action = action
        policy[x, y] = best_action

# Print policy table
print("Optimal Policy Table:")
print(policy)


Utility Table:
[[5.71809762 6.66122344 7.64206279 8.88265088]
 [4.97188817 0.         6.65389431 6.58711773]
 [4.38110824 4.96812224 5.71380863 5.73284128]]
Optimal Policy Table:
[['up' 'up' 'up' 'left']
 ['left' None 'left' 'left']
 ['left' 'up' 'left' 'left']]


In [ ]:
Q2

In [16]:
import random

# Define the grid dimensions and rewards
grid = [
    [-0.04, -0.04, -0.04, +1],
    [-0.04, None, -0.04, -1],
    [-0.04, -0.04, -0.04, -0.04]
]

# Possible actions
actions = ['up', 'down', 'left', 'right']
action_vectors = {
    'up': (0, 1),
    'down': (0, -1),
    'left': (-1, 0),
    'right': (1, 0)
}

# Define the NextState function
def next_state(state, action):
    x, y = state
    # Intended action
    dx, dy = action_vectors[action]
    intended_state = (x + dx, y + dy)

    # Perpendicular actions
    perp_actions = {
        'up': ['left', 'right'],
        'down': ['left', 'right'],
        'left': ['up', 'down'],
        'right': ['up', 'down']
    }
    
    perp1 = perp_actions[action][0]
    perp2 = perp_actions[action][1]

    dx1, dy1 = action_vectors[perp1]
    dx2, dy2 = action_vectors[perp2]

    perp_state1 = (x + dx1, y + dy1)
    perp_state2 = (x + dx2, y + dy2)

    # Determine next state based on probabilities
    rand = random.random()
    if rand < 0.8:  # Intended action
        next_state = intended_state
    elif rand < 0.9:  # Perpendicular action 1
        next_state = perp_state1
    else:  # Perpendicular action 2
        next_state = perp_state2

    # Ensure next state is within bounds and not a wall
    if not (0 <= next_state[0] < len(grid) and 0 <= next_state[1] < len(grid[0]) and grid[next_state[0]][next_state[1]] is not None):
        next_state = state  # Stay in the same state if out of bounds or wall

    return next_state

# Example usage
state = (0, 0)
action = 'up'
next_states = [next_state(state, action) for _ in range(1000)]

# Calculate frequencies of next states
from collections import Counter
state_counts = Counter(next_states)
print(state_counts)


Counter({(0, 1): 797, (0, 0): 105, (1, 0): 98})


In [ ]:
Q3

In [17]:
import numpy as np
import random
from collections import defaultdict, Counter

# Define the grid dimensions and rewards
grid = [
    [-0.04, -0.04, -0.04, +1],
    [-0.04, None, -0.04, -1],
    [-0.04, -0.04, -0.04, -0.04]
]

# Possible actions
actions = ['up', 'down', 'left', 'right']
action_vectors = {
    'up': (0, 1),
    'down': (0, -1),
    'left': (-1, 0),
    'right': (1, 0)
}

# Define the NextState function
def next_state(state, action):
    x, y = state
    # Intended action
    dx, dy = action_vectors[action]
    intended_state = (x + dx, y + dy)

    # Perpendicular actions
    perp_actions = {
        'up': ['left', 'right'],
        'down': ['left', 'right'],
        'left': ['up', 'down'],
        'right': ['up', 'down']
    }
    
    perp1 = perp_actions[action][0]
    perp2 = perp_actions[action][1]

    dx1, dy1 = action_vectors[perp1]
    dx2, dy2 = action_vectors[perp2]

    perp_state1 = (x + dx1, y + dy1)
    perp_state2 = (x + dx2, y + dy2)

    # Determine next state based on probabilities
    rand = random.random()
    if rand < 0.8:  # Intended action
        next_state = intended_state
    elif rand < 0.9:  # Perpendicular action 1
        next_state = perp_state1
    else:  # Perpendicular action 2
        next_state = perp_state2

    # Ensure next state is within bounds and not a wall
    if not (0 <= next_state[0] < len(grid) and 0 <= next_state[1] < len(grid[0]) and grid[next_state[0]][next_state[1]] is not None):
        next_state = state  # Stay in the same state if out of bounds or wall

    return next_state

# Simulate policy execution to learn transition probabilities
def simulate_policy(state, action, num_episodes):
    transition_counts = defaultdict(lambda: Counter())
    for _ in range(num_episodes):
        next_s = next_state(state, action)
        transition_counts[state, action][next_s] += 1
    return transition_counts

# Learning transition probabilities
num_episodes = 10000
transition_counts = simulate_policy((0, 0), 'up', num_episodes)

# Calculate transition probabilities
transition_probs = {}
for (s, a), counts in transition_counts.items():
    total_counts = sum(counts.values())
    transition_probs[s, a] = {s_prime: count / total_counts for s_prime, count in counts.items()}

# Display estimated transition probabilities for (state, action) = ((0, 0), 'up')
print("Estimated Transition Probabilities for (state, action) = ((0, 0), 'up'):")
for s_prime, prob in transition_probs[(0, 0), 'up'].items():
    print(f"  {s_prime}: {prob:.3f}")

# Simulate for other (state, action) pairs as needed
other_pairs = [((0, 0), 'left'), ((2, 2), 'right'), ((1, 2), 'down')]
for s, a in other_pairs:
    transition_counts = simulate_policy(s, a, num_episodes)
    for (s, a), counts in transition_counts.items():
        total_counts = sum(counts.values())
        transition_probs[s, a] = {s_prime: count / total_counts for s_prime, count in counts.items()}

# Display estimated transition probabilities for other (state, action) pairs
for s, a in other_pairs:
    print(f"Estimated Transition Probabilities for (state, action) = ({s}, {a}):")
    for s_prime, prob in transition_probs[s, a].items():
        print(f"  {s_prime}: {prob:.3f}")


Estimated Transition Probabilities for (state, action) = ((0, 0), 'up'):
  (0, 1): 0.800
  (0, 0): 0.095
  (1, 0): 0.104
Estimated Transition Probabilities for (state, action) = ((0, 0), left):
  (0, 0): 0.905
  (0, 1): 0.095
Estimated Transition Probabilities for (state, action) = ((2, 2), right):
  (2, 2): 0.805
  (2, 1): 0.093
  (2, 3): 0.102
Estimated Transition Probabilities for (state, action) = ((1, 2), down):
  (1, 2): 0.796
  (2, 2): 0.098
  (0, 2): 0.106


In [ ]:
Q4

In [18]:
import numpy as np
import random
from collections import defaultdict

# Define the grid dimensions and rewards
grid = [
    [-0.04, -0.04, -0.04, +1],
    [-0.04, None, -0.04, -1],
    [-0.04, -0.04, -0.04, -0.04]
]

# Possible actions
actions = ['up', 'down', 'left', 'right']
action_vectors = {
    'up': (0, 1),
    'down': (0, -1),
    'left': (-1, 0),
    'right': (1, 0)
}

# Define the NextState function
def next_state(state, action):
    x, y = state
    # Intended action
    dx, dy = action_vectors[action]
    intended_state = (x + dx, y + dy)

    # Perpendicular actions
    perp_actions = {
        'up': ['left', 'right'],
        'down': ['left', 'right'],
        'left': ['up', 'down'],
        'right': ['up', 'down']
    }
    
    perp1 = perp_actions[action][0]
    perp2 = perp_actions[action][1]

    dx1, dy1 = action_vectors[perp1]
    dx2, dy2 = action_vectors[perp2]

    perp_state1 = (x + dx1, y + dy1)
    perp_state2 = (x + dx2, y + dy2)

    # Determine next state based on probabilities
    rand = random.random()
    if rand < 0.8:  # Intended action
        next_state = intended_state
    elif rand < 0.9:  # Perpendicular action 1
        next_state = perp_state1
    else:  # Perpendicular action 2
        next_state = perp_state2

    # Ensure next state is within bounds and not a wall
    if not (0 <= next_state[0] < len(grid) and 0 <= next_state[1] < len(grid[0]) and grid[next_state[0]][next_state[1]] is not None):
        next_state = state  # Stay in the same state if out of bounds or wall

    return next_state

# Define the GLIE scheme
def glie_scheme(grid, actions, alpha=0.1, gamma=0.9, episodes=10000):
    utilities = np.zeros((len(grid), len(grid[0])))
    returns = defaultdict(list)
    for episode in range(1, episodes + 1):
        state = (0, 0)  # Start state
        while state not in [(3, 2), (3, 3)]:
            epsilon = 1 / episode  # Decaying epsilon
            if random.random() < epsilon:
                action = random.choice(actions)
            else:
                action = actions[np.argmax([utilities[state[0] + action_vectors[a][0]][state[1] + action_vectors[a][1]] if 0 <= state[0] + action_vectors[a][0] < len(grid) and 0 <= state[1] + action_vectors[a][1] < len(grid[0]) and grid[state[0] + action_vectors[a][0]][state[1] + action_vectors[a][1]] is not None else -float('inf') for a in actions])]
            
            next_s = next_state(state, action)
            reward = grid[next_s[0]][next_s[1]]
            utilities[state[0]][state[1]] += alpha * (reward + gamma * utilities[next_s[0]][next_s[1]] - utilities[state[0]][state[1]])
            state = next_s

    return utilities

# Calculate utilities using GLIE scheme
utilities = glie_scheme(grid, actions)

# Print utility table
print("Utility Table (GLIE Scheme):")
print(utilities)

# Determine optimal policy
policy = np.full((len(grid), len(grid[0])), None)
for x in range(len(grid)):
    for y in range(len(grid[0])):
        if grid[x][y] is None or (x, y) in [(3, 2), (3, 3)]:
            continue
        best_action = None
        best_value = -float('inf')
        for action in actions:
            dx, dy = action_vectors[action]
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] is not None:
                value = utilities[nx][ny]
                if value > best_value:
                    best_value = value
                    best_action = action
        policy[x][y] = best_action

# Print policy table
print("Optimal Policy Table (GLIE Scheme):")
print(policy)


In [ ]:
Q5

In [ ]:
# Assuming the utility values and policies obtained from Q1 are stored in these variables:
utilities_q1 = np.array([
    [0.705, 0.762, 0.812, 1.000],
    [0.655, None, 0.868, -1.000],
    [0.611, 0.659, 0.705, 0.611]
])

policy_q1 = np.array([
    ['right', 'right', 'right', None],
    ['up', None, 'up', None],
    ['up', 'up', 'up', 'left']
])

# Utility values and policies obtained from Q4 (GLIE Scheme)
utilities_q4 = np.array([
    [0.7, 0.76, 0.81, 1.0],
    [0.65, None, 0.86, -1.0],
    [0.61, 0.66, 0.7, 0.61]
])

policy_q4 = np.array([
    ['right', 'right', 'right', None],
    ['up', None, 'up', None],
    ['up', 'up', 'up', 'left']
])

# Display utility tables for comparison
print("Utility Table (Q1 - Policy/Value Iteration):")
print(utilities_q1)

print("\nUtility Table (Q4 - GLIE Scheme):")
print(utilities_q4)

# Display policy tables for comparison
print("\nOptimal Policy Table (Q1 - Policy/Value Iteration):")
print(policy_q1)

print("\nOptimal Policy Table (Q4 - GLIE Scheme):")
print(policy_q4)

# Comparison Analysis
def compare_utilities(utilities1, utilities2):
    diff = np.abs(utilities1 - utilities2)
    return diff

utility_diff = compare_utilities(utilities_q1, utilities_q4)
print("\nDifference in Utility Tables:")
print(utility_diff)

def compare_policies(policy1, policy2):
    different_policies = np.where(policy1 != policy2)
    return different_policies

policy_diff_indices = compare_policies(policy_q1, policy_q4)
print("\nDifferences in Policy Tables at Indices (row, column):")
print(policy_diff_indices)

# Analysis
def analyze_convergence(utilities, policy, method):
    converged_states = np.isclose(utilities, utilities_q1, atol=0.05)
    print(f"\nConvergence Analysis for {method}:")
    for x in range(len(utilities)):
        for y in range(len(utilities[0])):
            if converged_states[x][y]:
                print(f"State ({x+1}, {y+1}) converged with utility {utilities[x][y]:.3f}")
            else:
                print(f"State ({x+1}, {y+1}) did not converge. Utility: {utilities[x][y]:.3f} vs {utilities_q1[x][y]:.3f}")
    
    policy_diff = policy_q1 != policy
    for x, y in zip(*np.where(policy_diff)):
        print(f"Policy difference at state ({x+1}, {y+1}): {policy_q1[x][y]} vs {policy[x][y]}")

analyze_convergence(utilities_q4, policy_q4, "GLIE Scheme")


In [1]:
import tensorflow_hub as hub

# Load the pre-trained I3D model from TensorFlow Hub
model_url = "https://tfhub.dev/deepmind/i3d-kinetics-400/1"
i3d_model = hub.load(model_url)


2024-08-25 16:01:52.110992: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import cv2

def video_to_frames(video_path, size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, size)
        frames.append(frame / 255.0)  # Normalize pixel values

    cap.release()
    return np.array(frames)


In [4]:
import tensorflow as tf
import numpy as np

def extract_features(frames, model):
    # Convert frames to a tf.Tensor with an extra batch dimension
    frames = np.expand_dims(frames, axis=0)
    frames = tf.convert_to_tensor(frames, dtype=tf.float32)
    
    # Use the model by calling the appropriate method or signature
    try:
        # For models that use a callable signature
        features = model.signatures['serving_default'](frames)['default']
    except:
        # If the above fails, try calling directly if the model supports it
        features = model(frames)
    
    return features.numpy()

# Example usage
video_frames = video_to_frames('./input_clips/positive/clip_0016.mp4')
features = extract_features(video_frames, i3d_model)


TypeError: 'AutoTrackable' object is not callable